In [1]:
import boto3
from botocore.exceptions import ClientError
import json
from decimal import Decimal
from pprint import pprint

In [2]:
def create_table(self,table_name,dynamodb=None):
    try:
        self.__table__ = dynamodb.create_table(
            TableName = table_name,
            KeySchema = [
                {
                    'AttributeName' : 'device_id',
                    'KeyType' : 'HASH'
                },
                {
                    'AttributeName' : 'datacount',
                    'KeyType' : 'RANGE'
                }
            ],
            AttributeDefinitions = [
                {
                    'AttributeName' : 'device_id',
                    'AttributeType' : 'S'
                },
                {
                    'AttributeName' : 'datacount',
                    'AttributeType' : 'N'
                }
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 10,
                'WriteCapacityUnits': 10
            }
        )
    except ClientError as e:
        print(e.response['Error']['Message'])
    return device_table

In [3]:
class AwsDynamodb:
    def __init__(self,table_name,dynamodb=None):
        if not dynamodb:
            dynamodb = boto3.resource('dynamodb')
            
        self.__table__= dynamodb.Table(table_name)
        
        
    def get_data(self,device_id,datacount):
        self.device_id = device_id
        self.datacount = datacount
        try:
            response = self.__table__.get_item(Key={'device_id': self.device_id, 'datacount': self.datacount})
        except ClientError as e:
            print(e.response['Error']['Message'])
        else:
            return response['Item']
        
    def load_data_with_json(self,json_data,dynamodb=None):
        try:
            for device in Devices:
                device_table.put_item(Item = json_data)
                print("loading :", device['device_id'],device['datacount'])
        except ClientError as e:
            print(e.response['Error']['Message'])
            
    def load_data(self,key):
        try:
            response = self.__table__.put_item(Item={**key})
        except ClientError as e:
            print(e.response['Error']['Message'])
        else:
            return response
        
    def update_data(self,key,update_exp,exp_attribute,return_value):
        try:
            response = self.__table__.update_item(
                Key={**key},
                UpdateExpression=update_exp,
                ExpressionAttributeValues={**exp_attribute},
                ReturnValues=return_value)
            
        except ClientError as e:
            print(e.response['Error']['Message'])
        else:
            return response
        
    def delete_data(self,key,cond_exp,exp_attribute):
        try:
            response = self.__table__.delete_item(
                Key={**key},
                ConditionExpression=cond_exp,
                ExpressionAttributeValues={**exp_attribute})
        except ClientError as e:
            if e.response['Error']['Code']=='ConditionalCheckFailedException':
                print(e.response['Error']['Message'])
            else:
                raise
        else:
            return response

In [ ]:
if __name__ == '__main__':

    # create table 
    device_table = create_table()
    print("table status:", device_table.table_status)

In [ ]:
#create object of class pass tableName as parameter
devices = AwsDynamodb('Device_Data')

In [ ]:
if __name__ == '__main__':
    #load data with json file
    with open("device.json") as json_data:
        device_list = json.load(json_data,parse_float=Decimal)
    load_data(device_list)

In [ ]:
if __name__ == '__main__':
    #load data manually
    device_id="10002"
    datacount=3
    timestamp="1612522800"
    temperature1= "23.74"
    temperature2= "32.56"
    temperature3= "12.43"
    temperature4= "44.74"
    temperature5= "12.74"

    receive_response = devices.load_data({
                        'device_id': device_id,
                        'datacount': datacount,
                        'info': {
                            'info_timestamp': timestamp,
                            'temperature1': temperature1,
                            'temperature2': temperature2,
                            'temperature3': temperature3,
                            'temperature4': temperature4,
                            'temperature5': temperature5
                }})
    pprint(receive_response)

In [ ]:
if __name__ == '__main__':
    #get data with primary key
    receive_data = devices.get_data('10002',2)
    pprint(receive_data)

In [ ]:
if __name__ == '__main__':
    #update data 
    key={'device_id':'10001','datacount':1}
    update_exp='set info.info_timestamp=time,info.temperature1=:t1,info.temperature2=:t2, info.temperature3=:t3, info.temperature4=:t4, info.temperature5=:t5'
    exp_attribute = {
        'time':"1612522800",
        ':t1': "50.1",
        ':t2': "51.1",
        ':t3': "52.1",
        ':t4': "53.1",
        ':t5': "54.1"
    }
    return_value = "UPDATED_NEW"
    update_response = devices.update_data(key,update_exp,exp_attribute,return_value)
    
    pprint(update_response['Attributes'])

In [ ]:
if __name__ == '__main__':
    #delete data 
    key={'device_id':'10001','datacount':1}
    cond_exp = 'info.info_timestamp <= :value'
    exp_attribute = {':value':'1612522800'}

    delete_response = devices.delete_data(key,cond_exp,exp_attribute)
    pprint(delete_response)